<a href="https://colab.research.google.com/github/KashyapTamakuwala/Kaggle/blob/main/digit_identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kashyaptamakuwala","key":"e217cd8c0de72483dd421481ab2cc462"}'}

In [2]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [3]:
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle competitions download -c digit-recognizer

 82% 5.00M/6.09M [00:00<00:00, 46.1MB/s]
100% 6.09M/6.09M [00:00<00:00, 39.0MB/s]
 98% 9.00M/9.16M [00:00<00:00, 21.0MB/s]
100% 9.16M/9.16M [00:00<00:00, 20.7MB/s]
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 79.0MB/s]


In [12]:
import numpy as np
import warnings
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd


from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
from keras.callbacks import EarlyStopping

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

pd.set_option('display.max_rows', 1000)
warnings.filterwarnings("ignore")



train_data = pd.read_csv('/content/train.csv.zip')
test_data = pd.read_csv('/content/test.csv.zip')

Found GPU at: /device:GPU:0


In [13]:
#reshaping image and normalizing the value 
IMG_SIZE = 28

x_train = train_data.iloc[:,1:]
x_train = x_train.values.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_train = train_data.iloc[:,0]
x_test = test_data
x_test = x_test.values.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
x_train = x_train/255.0
x_test = x_test/255.0

In [14]:
#spliting data in training and validation set
X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state = 42)
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1)
datagen.fit(X_train)
print(datagen)

In [15]:
earlystopping = EarlyStopping(monitor ="val_accuracy",
                              mode = 'auto', patience = 30,
                              restore_best_weights = True)


# defining model
model = Sequential()

#conv2d(nooffilter,sizeoffilter,inputshapeofdata)
model.add(Conv2D(128, (3, 3), input_shape = x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
# model.add(Dropout(0.2))

model.add(Conv2D(512, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
# model.add(Dropout(0.2))


model.add(Flatten())


model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(10, activation='softmax'))


model.compile(optimizer = 'adam',
  loss = 'sparse_categorical_crossentropy',
  metrics = ['accuracy'])

EPOCHS = 2000
BATCH_SIZE=32

#training model

history = model.fit(datagen.flow(X_train, y_train), epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_val, y_val), callbacks=[earlystopping])


Epoch 1/2000
1182/1182 [==============================] - 24s 20ms/step - loss: 0.2292 - accuracy: 0.9275 - val_loss: 0.0984 - val_accuracy: 0.9712
Epoch 2/2000
1182/1182 [==============================] - 24s 20ms/step - loss: 0.1145 - accuracy: 0.9647 - val_loss: 0.0550 - val_accuracy: 0.9805
Epoch 3/2000
1182/1182 [==============================] - 24s 20ms/step - loss: 0.0954 - accuracy: 0.9700 - val_loss: 0.0359 - val_accuracy: 0.9890
Epoch 4/2000
1182/1182 [==============================] - 24s 20ms/step - loss: 0.0834 - accuracy: 0.9746 - val_loss: 0.0333 - val_accuracy: 0.9886
Epoch 5/2000
1182/1182 [==============================] - 24s 20ms/step - loss: 0.0727 - accuracy: 0.9765 - val_loss: 0.0923 - val_accuracy: 0.9748
Epoch 6/2000
1182/1182 [==============================] - 24s 20ms/step - loss: 0.0707 - accuracy: 0.9780 - val_loss: 0.0307 - val_accuracy: 0.9898
Epoch 7/2000
1182/1182 [==============================] - 24s 20ms/step - loss: 0.0619 - accuracy: 0.9805 - val_

In [17]:
#prediction onn test data
predictions = model.predict([x_test])
solutions = []
for i in range(len(predictions)):
    solutions.append(np.argmax(predictions[i]))


#saving prediction in a csv file
final = pd.DataFrame()
final['ImageId']=[i+1 for i in test_data.index]
final['Label']=solutions
final.to_csv('/content/submission.csv', index=False)